In [31]:
import pandas as pd
import numpy as np 
import emoji
from ast import literal_eval
import regex
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
import re
import string
import nltk 
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.casual import TweetTokenizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.stem.isri import ISRIStemmer
import pyarabic.araby as araby
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from pylab import rcParams
from matplotlib import rc
import joblib
from transformers import AutoTokenizer, AutoModel


In [32]:
Nile = pd.read_csv("NileUniversity_Emotional11.csv")
Nile.head(10)

,ID,text,label
0,1,الاوليمبياد الجايه هكون لسه ف الكليه ..,none
1,2,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...,anger
2,3,كتنا نيله ف حظنا الهباب xD,sadness
3,4,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,joy
4,5,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...,none
5,6,دلوقتي عندها اربع سنين .... ومحدش يقدر يكلمها ...,surprise
6,7,الحب يسلبها كيدها يرجعها الي براءتها الاولي وي...,love
7,8,she3er: شوفي بـ عيني امنيات الغلابه واحلام ثلث...,sympathy
8,9,86: صوتك هو حبيبي الثالث بعدك وبعد عيونك ~...,love
9,10,عمر عسر اتغلب وخرج وبيكمل مسيره مصر الفاشله في...,sadness


In [33]:
Nile.tail()

,ID,text,label
10060,10061,2222: يلا يا جماعه حفله عمرو دياب خلصت نريح شو...,sadness
10061,10062,Mohamed5: اييييه دااا 😲😲 اوزيييل❤,surprise
10062,10063,عملتلها ريتويت بمناسبه ساره بتاعه الاوليمبياد 😃,none
10063,10064,وعليك قبلنا يانجم النجوم ياعندليب الحب والاحساس,joy
10064,10065,AlHamad يطلع ننهم كل شي سيء ووضيع كل خساسه الع...,anger


In [34]:
Nile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10065 entries, 0 to 10064
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      10065 non-null  int64 
 1   text    10064 non-null  object
 2   label   10065 non-null  object
dtypes: int64(1), object(2)
memory usage: 236.0+ KB


# Check nulls

In [35]:
Nile[Nile.text.isnull()]

,ID,text,label
8619,8620,NaN,love


In [36]:
Nile.dropna(inplace=True)

In [37]:
Nile.isnull().sum()

ID       0
text     0
label    0
dtype: int64

# Replace emoticon with emoji

In [38]:
 emoji = {
        ":)" : "🙂",
        ":(" : "🙁",
        "xD" : "😆",
        ":=(": "😭",
        ":'(": "😢",
        ":'‑(": "😢",
        "XD" : "😂",
        ":D" : "🙂",
        "♬" : "موسيقي",
        "♡" : "❤",
        "☻"  : "🙂",
        }

In [39]:
def emoticon(text):
    for i in range (len(text)):
        message = text.iloc[i]
        seperarate_word = message.split(' ')
        Nile.text.iloc[i] = ""

        for word in seperarate_word:
            text.iloc[i] += emoji.get(word, word) + " "

In [40]:
emoticon(Nile.text)

D:\python\anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [41]:
Nile.text[2]

'كتنا نيله ف حظنا الهباب 😆 '

# know the frequent emoji

In [42]:
import emoji
#to count the number of emojis in the dataset
def emoji_counter(sentence):
    return emoji.emoji_count(sentence)

In [43]:
import emoji
Nile['emoji_count'] = Nile['text'].apply(lambda x: emoji_counter(x))

In [44]:
Nile

,ID,text,label,emoji_count
0,1,الاوليمبياد الجايه هكون لسه ف الكليه ..,none,0
1,2,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...,anger,0
2,3,كتنا نيله ف حظنا الهباب 😆,sadness,1
3,4,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,joy,0
4,5,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...,none,0
...,...,...,...,...
10060,10061,2222: يلا يا جماعه حفله عمرو دياب خلصت نريح شو...,sadness,0
10061,10062,Mohamed5: اييييه دااا 😲😲 اوزيييل❤,surprise,3
10062,10063,عملتلها ريتويت بمناسبه ساره بتاعه الاوليمبياد 😃,none,1
10063,10064,وعليك قبلنا يانجم النجوم ياعندليب الحب والاحساس,joy,0


In [45]:
emojis_data = Nile[Nile['emoji_count'] > 0]

In [46]:
emojis_data.reset_index(drop=True,inplace=True)

In [47]:
emojis_data

,ID,text,label,emoji_count
0,3,كتنا نيله ف حظنا الهباب 😆,sadness,1
1,13,يا ويل قلبي منه ي كثر ما اغليه ويا كثر ماله ف...,love,1
2,19,المفروض يلغوها من الاوليمبياد وندخل الاسكواش م...,none,1
3,20,25jan اعوذ بالله مايسوي هذا الرز اللي تحكوا ...,sympathy,1
4,25,بعثه مصر في الاوليمبياد رايحه تتفرج زيها زينا 😂😂,none,2
...,...,...,...,...
2085,10058,للامانه كنت متعاطفه مع هيلاري كثير لانها حرمه ...,sympathy,1
2086,10059,كنت خايفه من هاد اليوم 💔💔💔 فاقولنا يا جدك الج...,fear,3
2087,10060,خايفه نبدا مذاكره ونعمل حاجات مفيده ونسيب الحا...,fear,1
2088,10062,Mohamed5: اييييه دااا 😲😲 اوزيييل❤,surprise,3


In [48]:
emojis_data.text.iloc[0]

'كتنا نيله ف حظنا الهباب 😆 '

In [49]:
def extract_emoji(text):
    
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)
    
    return emoji_list

In [50]:
emojis_data['emoji'] = emojis_data['text'].apply(lambda x: extract_emoji(x))
emojis_data['emoji'] = emojis_data['emoji'].apply(lambda x: ' '.join([word for word in x ]))

<ipython-input-50-c031bf97cbf4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emojis_data['emoji'] = emojis_data['text'].apply(lambda x: extract_emoji(x))
<ipython-input-50-c031bf97cbf4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emojis_data['emoji'] = emojis_data['emoji'].apply(lambda x: ' '.join([word for word in x ]))


In [51]:
emojis_data

,ID,text,label,emoji_count,emoji
0,3,كتنا نيله ف حظنا الهباب 😆,sadness,1,😆
1,13,يا ويل قلبي منه ي كثر ما اغليه ويا كثر ماله ف...,love,1,❤
2,19,المفروض يلغوها من الاوليمبياد وندخل الاسكواش م...,none,1,😂
3,20,25jan اعوذ بالله مايسوي هذا الرز اللي تحكوا ...,sympathy,1,💔
4,25,بعثه مصر في الاوليمبياد رايحه تتفرج زيها زينا 😂😂,none,2,😂 😂
...,...,...,...,...,...
2085,10058,للامانه كنت متعاطفه مع هيلاري كثير لانها حرمه ...,sympathy,1,😁
2086,10059,كنت خايفه من هاد اليوم 💔💔💔 فاقولنا يا جدك الج...,fear,3,💔 💔 💔
2087,10060,خايفه نبدا مذاكره ونعمل حاجات مفيده ونسيب الحا...,fear,1,😂
2088,10062,Mohamed5: اييييه دااا 😲😲 اوزيييل❤,surprise,3,😲 😲 ❤


In [52]:
type(emojis_data.emoji.iloc[0])

str

In [53]:
pd.Series(' '.join(emojis_data['emoji']).split()).value_counts()

😂     737
💔     282
🙂     171
❤     170
❤️    151
     ... 
🐣       1
🙌       1
❌       1
👊🏽      1
🎀       1
Length: 275, dtype: int64

In [54]:
Nile.drop("emoji_count", axis=1, inplace=True)

In [55]:
Nile

,ID,text,label
0,1,الاوليمبياد الجايه هكون لسه ف الكليه ..,none
1,2,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...,anger
2,3,كتنا نيله ف حظنا الهباب 😆,sadness
3,4,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,joy
4,5,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...,none
...,...,...,...
10060,10061,2222: يلا يا جماعه حفله عمرو دياب خلصت نريح شو...,sadness
10061,10062,Mohamed5: اييييه دااا 😲😲 اوزيييل❤,surprise
10062,10063,عملتلها ريتويت بمناسبه ساره بتاعه الاوليمبياد 😃,none
10063,10064,وعليك قبلنا يانجم النجوم ياعندليب الحب والاحساس,joy


# Replace emojis with arabic words

In [56]:
 emojis = {
        "🙂":"يبتسم",
        "😂":"يضحك",
        "💔":"قلب حزين",
        "🙂":"يبتسم",
        "❤️":"حب",
        "❤":"حب",
        "😍":"حب",
        "😭":"يبكي",
        "😢":"حزن",
        "😔":"حزن",
        "♥":"حب",
        "💜":"حب",
        "😅":"يضحك",
        "🙁":"حزين",
        "💕":"حب",
        "💙":"حب",
        "😞":"حزين",
        "😊":"سعادة",
        "👏":"يصفق",
        "👌":"احسنت",
        "😴":"ينام",
        "😀":"يضحك",
        "😌":"حزين",
        "🌹":"وردة",
        "🙈":"حب",
        "😄":"يضحك",
        "😐":"محايد",
        "✌":"منتصر",
        "✨":"نجمه",
        "🤔":"تفكير",
        "😏":"يستهزء",
        "😒":"يستهزء",
        "🙄":"ملل",
        "😕":"عصبية",
        "😃":"يضحك",
        "🌸":"وردة",
        "😓":"حزن",
        "💞":"حب",
        "💗":"حب",
        "😑":"منزعج",
        "💭":"تفكير",
        "😎":"ثقة",
        "💛":"حب",
        "😩":"حزين",
        "💪":"عضلات",
        "👍":"موافق",
        "🙏🏻":"رجاء طلب",
        "😳":"مصدوم",
        "👏🏼":"تصفيق",
        "🎶":"موسيقي",
        "🌚":"صمت",
        "💚":"حب",
        "🙏":"رجاء طلب",
        "💘":"حب",
        "🍃":"سلام",
        "☺":"يضحك",
        "🐸":"ضفدع",
        "😶":"مصدوم",
        "✌️":"مرح",
        "✋🏻":"توقف",
        "😉":"غمزة",
        "🌷":"حب",
        "🙃":"مبتسم",
        "😫":"حزين",
        "😨":"مصدوم",
        "🎼 ":"موسيقي",
        "🍁":"مرح",
        "🍂":"مرح",
        "💟":"حب",
        "😪":"حزن",
        "😆":"يضحك",
        "😣":"استياء",
        "☺️":"حب",
        "😱":"كارثة",
        "😁":"يضحك",
        "😖":"استياء",
        "🏃🏼":"يجري",
        "😡":"غضب",
        "🚶":"يسير",
        "🤕":"مرض",
        "‼️":"تعجب",
        "🕊":"طائر",
        "👌🏻":"احسنت",
        "❣":"حب",
        "🙊":"مصدوم",
        "💃":"سعادة مرح",
        "💃🏼":"سعادة مرح",
        "😜":"مرح",
        "👊":"ضربة",
        "😟":"استياء",
        "💖":"حب",
        "😥":"حزن",
        "🎻":"موسيقي",
        "✒":"يكتب",
        "🚶🏻":"يسير",
        "💎":"الماظ",
        "😷":"وباء مرض",
        "☝":"واحد",
        "🚬":"تدخين",
        "💐" : "ورد",
        "🌞" : "شمس",
        "👆" : "الاول",
        "⚠️" :"تحذير",
        "🤗" : "احتواء",
        "✖️": "غلط",
        "📍"  : "مكان",
        "👸" : "ملكه",
        "👑" : "تاج",
        "✔️" : "صح",
        "💌": "قلب",
        "😲" : "مندهش",
        "💦": "ماء",
        "🚫" : "خطا",
        "👏🏻" : "برافو",
        "🏊" :"يسبح",
        "👍🏻": "تمام",
        "⭕️" :"دائره كبيره",
        "🎷" : "ساكسفون",
        "👋": "تلويح باليد",
        "✌🏼": "علامه النصر",
        "🌝":"مبتسم",
        "➿"  : "عقده مزدوجه",
        "💪🏼" : "قوي",
        "📩":  "تواصل معي",
        "☕️": "قهوه",
        "😧" : "قلق و صدمة",
        "🗨": "رسالة",   
        "❗️" :"تعجب",
        "🙆🏻": "اشاره موافقه",
        "👯" :"اخوات",
        "©" :  "رمز",
        "👵🏽" :"سيده عجوزه",
        "🐣": "كتكوت",  
        "🙌": "تشجيع",
        "🙇": "شخص ينحني",
        "👐🏽":"ايدي مفتوحه",    
        "👌🏽": "بالظبط",
        "⁉️" : "استنكار",
        "⚽️": "كوره",
        "🕶" :"حب",
        "🎈" :"بالون",    
        "🎀":    "ورده",
        "💵":  "فلوس",   
        "😋":  "جائع",
        "😛":  "يغيظ",
        "😠":  "غاضب",
        "✍🏻":  "يكتب",
        "🌾":  "ارز",
        "👣":  "اثر قدمين",
        "❌":"رفض",
        "🍟":"طعام",
        "👬":"صداقة",
        "🐰":"ارنب",
        "☂":"مطر",
        "⚜":"مملكة فرنسا",
        "🐑":"خروف",
        "🗣":"صوت مرتفع",
        "👌🏼":"احسنت",
        "☘":"مرح",
        "😮":"صدمة",
        "😦":"قلق",
        "⭕":"الحق",
        "✏️":"قلم",
        "ℹ":"معلومات",
        "🙍🏻":"رفض",
        "⚪️":"نضارة نقاء",
        "🐤":"حزن",
        "💫":"مرح",
        "💝":"حب",
        "🍔":"طعام",
        "❤︎":"حب",
        "✈️":"سفر",
        "🏃🏻‍♀️":"يسير",
        "🍳":"ذكر",
        "🎤":"مايك غناء",
        "🎾":"كره",
        "🐔":"دجاجة",
        "🙋":"سؤال",
        "📮":"بحر",
        "💉":"دواء",
        "🙏🏼":"رجاء طلب",
        "💂🏿 ":"حارس",
        "🎬":"سينما",
        "♦️":"مرح",
        "💡":"قكرة",
        "‼":"تعجب",
        "👼":"طفل",
        "🔑":"مفتاح",
        "♥️":"حب",
        "🕋":"كعبة",
        "🐓":"دجاجة",
        "💩":"معترض",
        "👽":"فضائي",
        "☔️":"مطر",
        "🍷":"عصير",
        "🌟":"نجمة",
        "☁️":"سحب",
        "👃":"معترض",
        "🌺":"مرح",
        "🔪":"سكينة",
        "♨":"سخونية",
        "👊🏼":"ضرب",
        "✏":"قلم",
        "🚶🏾‍♀️":"يسير",
        "👊":"ضربة",
        "◾️":"وقف",
        "😚":"حب",
        "🔸":"مرح",
        "👎🏻":"لا يعجبني",
        "👊🏽":"ضربة",
        "😙":"حب",
        "🎥":"تصوير",
        "👉":"جذب انتباه",
        "👏🏽":"يصفق",
        "💪🏻":"عضلات",
        "🏴":"اسود",
        "🔥":"حريق",  
        "😬":"عدم الراحة",   
        "👊🏿":"يضرب",    
        "🌿":"ورقه شجره",     
        "✋🏼":"كف ايد",    
        "👐":"ايدي مفتوحه",      
        "☠️":"وجه مرعب",     
        "🎉":"يهنئ",      
        "🔕" :"صامت",
        "😿":"وجه حزين",      
        "☹️":"وجه يائس",     
        "😘" :"حب",     
        "😰" :"خوف و حزن",
        "🌼":"ورده",      
        "💋":  "بوسه",
        "👇":"لاسفل",     
        "❣️":"حب",     
        "🎧":"سماعات",
        "📝":"يكتب",      
        "😇":"دايخ",      
        "😈":"رعب",      
        "🏃":"يجري",      
        "✌🏻":"علامه النصر",    
        "🔫":"يضرب",      
        "❗️":"تعجب",
        "👎":"غير موافق",      
        "🔐":"قفل",      
        "👈":"لليمين",
        "™":"رمز",    
        "🚶🏽":"يتمشي",    
        "😯":"متفاجأ",  
        "✊":"يد مغلقه",    
        "😻":"اعجاب",    
        "🙉" :"قرد",    
        "👧":"طفله صغيره",     
        "🔴":"دائره حمراء",      
        "💪🏽":"قوه",     
        "💤":"ينام",     
        "👀":"ينظر",     
        "✍🏻":"يكتب",  
        "❄️":"تلج",
        "💀":"رعب",   
        "😤":"وجه عابس",      
        "🖋":"قلم",      
        "🎩":"كاب",      
        "☕️":"قهوه",     
        "😹":"ضحك",     
        "💓":"حب",      
        "☄️ ":"نار",     
        "👻":"رعب",
        }
        
                
        

In [57]:
def emojii(text):
    for i in range (len(text)):
        message = text.iloc[i]
        seperarate_word = regex.findall(r'\X', message)
        text.iloc[i] = ""

        for word in seperarate_word:
            if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
                    text.iloc[i] += " " + emojis.get(word, word) + " "
            else:
                    text.iloc[i] +=  emojis.get(word, word) + ""

In [58]:
emojii(Nile.text)

D:\python\anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [59]:
Nile.text[10017]

'كـثر شـوقـي لـهہ ونـقـص برودي الم ذگـريـاتهہ لـلحـيـن يجرحـنـي قلب حزين  !  غــرام قلب حزين  '

In [60]:
df = pd.DataFrame(Nile) 
  
# saving the dataframe 
df.to_csv(r'D:\downlowds google\replace_emoji.csv', index=False)